## Contextual Compression Retriever란?
- 검색된 문서에서 쿼리와 관련된 정보만을 압축하여 추출하는 Retriever
- 기본 Retriever가 검색한 문서를 Document Compressor가 압축하여 품질을 향상
- 불필요한 정보를 제거하여 LLM에 전달되는 정보의 노이즈 개선

**기존 RAG 문제점**
1. **관련 없는 정보**: 청크에 쿼리와 무관한 정보가 포함되어 LLM을 혼란시킴
2. **토큰 낭비**: 긴 문서가 프롬프트 공간을 차지하여 비용 증가
3. **정확도 저하**: 불필요한 정보로 인한 응답 품질 저하

**핵심 개념**
- **Base Retriever**: 기본 문서 검색을 담당하는 Retriever (VectorStore 등)
- **Document Compressor**: 검색된 문서를 압축/필터링하는 구성요소


실습할 문서 만들기

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/[AI.GOV_해외동향]_2025-1호.pdf")

document = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],  # 문단 → 줄 → 문장 → 단어 순
    chunk_size=300,    
    chunk_overlap=30
)

chunks = recursive_splitter.split_documents(document)

print(f"총 {len(chunks)}개 청크 생성\n")

for i, chunk in enumerate(chunks, 1):
    chunk_length = len(chunk.page_content)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk.page_content}'")

임베딩 객체 생성

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

Vector Store 생성

In [ ]:
from langchain_community.vectorstores import FAISS

# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(
    documents=chunks, 
    embedding=embeddings
)

기본 VectorStore Retriever 테스트

In [ ]:
# 기본 VectorStore Retriever 생성
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 기본 검색 테스트
question = "각국의 인프라 투자 상황에 대해 알려주세요"
docs = base_retriever.invoke(question)

print(f"=== 기본 검색 결과 (총 {len(docs)}개) ===")

for i, doc in enumerate(docs):
    print(f"문서 {i+1}:")
    print(f"내용: {doc.page_content}")

### LLMChainExtractor를 사용한 Contextual Compression

**LLMChainExtractor**
- LLM을 이용하여 문서나 텍스트로부터 정보(예: 키워드, 요약, 질문 등)를 추출하는 유틸
- LLM + Prompt + OutputParser를 묶어서, 텍스트에서 원하는 정보를 추출 할 수도 있음.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.0)

# LLMChainExtractor 생성
# 문서에서 쿼리와 관련된 정보만을 추출
compressor = LLMChainExtractor.from_llm(llm)

# ContextualCompressionRetriever 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# 압축된 검색 결과
compressed_docs = compression_retriever.invoke(question)

In [ ]:

print(f"LLMChainExtractor 압축 결과 (총 {len(compressed_docs)}개)")
for i, doc in enumerate(compressed_docs):
    print(i)
    print(f"내용: {doc.page_content}")